In [1]:
import numpy as np
import pandas as pd
from scipy.stats import binned_statistic_2d
import re
from datetime import datetime
import itertools
import matplotlib.pyplot as plt

### Прочитаем исходные данные
На примере данных на май 2016 напишем код, который очистит и предобработает данные.

In [105]:
raw_file = pd.read_csv('/Users/shrlq/Documents/python_yandex/course 6/week_3/raw data/yellow_tripdata_2016-05.csv', 
                       sep=',')
raw_file.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,40.730099,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,40.765469,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,40.744633,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,40.733002,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,40.737564,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76


In [106]:
raw_file.columns = raw_file.columns.str.strip() # удалим пробелы в названиях колонок

### Отфильтруем данные
А именно - удалим поездки с:
- координатами начала, не попадающими в прямоугольник Нью-Йорка
- нулевой длительностью
- нулевым количеством пассажиров
- нулевым расстоянием поездки по счётчику


In [107]:
# названия колонок в разных файлах меняются, поэтому используем индексы
print ('Кол-во строк в исходной таблице:' + ' ' + str(raw_file.shape[0]) )
raw_file_f = raw_file.loc[sum([raw_file[raw_file.columns[5]] < -74.25559, raw_file[raw_file.columns[5]] > -73.70001]) == 0, :]
print ('После фильтрации по долготе:' + ' ' + str(raw_file_f.shape[0]) )

raw_file_f = raw_file_f.loc[sum([raw_file_f[raw_file.columns[6]] < 40.49612, raw_file_f[raw_file.columns[6]] > 40.91553]) == 0, :]
print ('После фильтрации по широте:' + ' ' + str(raw_file_f.shape[0]) )

raw_file_f = raw_file_f.loc[raw_file_f[raw_file.columns[1]] != raw_file_f[raw_file.columns[2]], :]
print ('После фильтрации по времени поездки:' + ' ' + str(raw_file_f.shape[0]) )

raw_file_f = raw_file_f.loc[raw_file_f[raw_file.columns[3]] != 0, :]
print ('После фильтрации по кол-ву пассажиров поездки:' + ' ' + str(raw_file_f.shape[0]) )

raw_file_f = raw_file_f.loc[raw_file_f[raw_file.columns[4]] != 0, :]
print ('После фильтрации по расстоянию по счетку поездки:' + ' ' + str(raw_file_f.shape[0]) )


Кол-во строк в исходной таблице: 11836853
После фильтрации по долготе: 11687416
После фильтрации по широте: 11686529
После фильтрации по времени поездки: 11675138
После фильтрации по кол-ву пассажиров поездки: 11674671
После фильтрации по расстоянию по счетку поездки: 11626521


### Добавим идентификаторы зон
Разделим все поездки на 2500 зон в зависимости от их геокоординат.
Для этого нам понадобится файл regions.csv, в котором сделан маппинг идентификаторов зон с координатами их границ.

In [108]:
regions_data = pd.read_csv('regions.csv', sep = ';')
regions_data.head()

,region,west,east,south,north
0,1,-74.25559,-74.244478,40.496120,40.504508
1,2,-74.25559,-74.244478,40.504508,40.512896
2,3,-74.25559,-74.244478,40.512896,40.521285
3,4,-74.25559,-74.244478,40.521285,40.529673
4,5,-74.25559,-74.244478,40.529673,40.538061


Поделим Нью-Йорк по широте и долготе на 50 отрезков по оси X и 50 отрезков по оси Y (согласно маппингу).

In [109]:
eastwest = np.unique([regions_data['west'], regions_data['east']])
southnorth = np.unique([regions_data['south'], regions_data['north']])
raw_file_binned = binned_statistic_2d(x=raw_file_f[raw_file.columns[5]], y=raw_file_f[raw_file.columns[6]], values=None, statistic = 'count', 
                          bins=[eastwest, southnorth],
                         expand_binnumbers=True)

print (raw_file_binned.binnumber) # номера отрезков по оси X и Y для поездок из таблицы с отфильтрованными данными
print (raw_file_binned.binnumber.shape) # совпадает с общим кол-вом поездок в таблице с отфильтрованными данными

[[25 24 24 ... 23 25 28]
 [33 30 30 ... 27 32 11]]
(2, 11626521)


In [110]:
bins = [[x, y] for x, y in zip(raw_file_binned.binnumber[0, :], raw_file_binned.binnumber[1, :])]
bins_df = pd.DataFrame(data = bins, columns=['binX', 'binY'])
bins_df.head()

,binX,binY
0,25,33
1,24,30
2,24,30
3,24,23
4,23,30


Для каждой поездки в таблице добавим номер отрезка по оси X и по оси Y, соответствующей зоне начала поездки.

In [111]:
raw_file_f.index = bins_df.index # необходимо, так как индексы отфильтрованной таблице унаследованы от исходной таблицы и отличаются от bins_df
raw_file_c = pd.concat(objs=[raw_file_f, bins_df], axis=1)
raw_file_c.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,binX,binY
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,...,1,15.0,0.5,0.5,1.50,0.0,0.3,17.80,25,33
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,...,1,7.5,0.5,0.5,0.88,0.0,0.3,9.68,24,30
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,...,1,6.5,0.5,0.5,1.56,0.0,0.3,9.36,24,30
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,...,1,17.0,0.5,0.5,3.66,0.0,0.3,21.96,24,23
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,...,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76,23,30


На аналогичные отрезки разобьем файл с идентификаторами зон, чтобы получить соответствие между идентификатором зоны и номерам отрезков по оси X и Y.

In [112]:
regions_binned = binned_statistic_2d(x=regions_data['west'], y=regions_data['south'], values=regions_data['region'], 
                          bins=[eastwest, southnorth],
                         expand_binnumbers=True)
regions_binned_ind = np.matrix([list(map(int, a)) for a in regions_binned.statistic])

print (regions_binned_ind) # таблица с идентификаторами регионов, соответствующим квадратам (размер 50*50)
print (regions_binned.binnumber) # номера отрезков по оси X и Y, которым соответствует значение идентификатора региона
print (regions_binned.binnumber.shape) # совпадает с общим кол-вом регионов в таблице с идентификаторами регионов


[[   1    2    3 ...   48   49   50]
 [  51   52   53 ...   98   99  100]
 [ 101  102  103 ...  148  149  150]
 ...
 [2351 2352 2353 ... 2398 2399 2400]
 [2401 2402 2403 ... 2448 2449 2450]
 [2451 2452 2453 ... 2498 2499 2500]]
[[ 1  1  1 ... 50 50 50]
 [ 1  2  3 ... 48 49 50]]
(2, 2500)


Ниже полученное соответствие:

In [113]:
reg_ids = [[x, y, regions_binned_ind[x-1, y-1]] for x, y in zip(regions_binned.binnumber[0, :], regions_binned.binnumber[1, :])]
reg_df = pd.DataFrame(data = reg_ids, columns=['binX', 'binY', 'Reg_ID'])
reg_df.head()

,binX,binY,Reg_ID
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,1,5,5


С помощью номеров отрезков (колоки binX и binY) мы можем сопоставить данные о поездках с идентификатором зоны, что мы и сделаем, добавив колонку  Reg_ID в основную таблицу.
Также сразу преобразуем временные данные, отдельно выделив часы, которые в дальнейшем понадобятся для агрегации.

In [114]:
raw_file_add = pd.merge(left = raw_file_c, right = reg_df, how = 'left', on =['binX', 'binY'])
raw_file_add['tpep_pickup_hour'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H') for x in raw_file_add[raw_file.columns[1]]]
raw_file_add['tpep_dropoff_hour'] = [datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d %H') for x in raw_file_add[raw_file.columns[2]]]

raw_file_add.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,binX,binY,Reg_ID,tpep_pickup_hour,tpep_dropoff_hour
0,1,2016-05-01 00:00:00,2016-05-01 00:17:31,1,3.60,-73.985901,40.768040,1,N,-73.983986,...,0.5,1.50,0.0,0.3,17.80,25,33,1233,2016-05-01 00,2016-05-01 00
1,2,2016-05-01 00:00:00,2016-05-01 00:07:31,1,1.68,-73.991577,40.744751,1,N,-73.975700,...,0.5,0.88,0.0,0.3,9.68,24,30,1180,2016-05-01 00,2016-05-01 00
2,2,2016-05-01 00:00:00,2016-05-01 00:07:01,6,1.09,-73.993073,40.741573,1,N,-73.980995,...,0.5,1.56,0.0,0.3,9.36,24,30,1180,2016-05-01 00,2016-05-01 00
3,2,2016-05-01 00:00:00,2016-05-01 00:19:47,1,4.21,-73.991943,40.684601,1,N,-74.002258,...,0.5,3.66,0.0,0.3,21.96,24,23,1173,2016-05-01 00,2016-05-01 00
4,2,2016-05-01 00:00:00,2016-05-01 00:06:39,1,0.56,-74.005280,40.740192,1,N,-73.997498,...,0.5,1.46,0.0,0.3,8.76,23,30,1130,2016-05-01 00,2016-05-01 00


### Агрегируем данные по часам и зонам поездок (по посадке пассажиров)
Таким образом, чтобы получить следующие колонки:
- время
- идентификатор зоны
- количество поездок
- средняя дистанция
- средний платеж

In [115]:
# Агрегируем данные по часам и зонам поездок (по посадке пассажиров)
raw_file_add['pickup'] = ''
raw_file_gr = raw_file_add.groupby(by = ['tpep_pickup_hour', 'Reg_ID'], as_index=False)
raw_file_ag = raw_file_gr['pickup'].count()
raw_file_ag = raw_file_ag.merge(raw_file_gr['trip_distance'].median(), on=['tpep_pickup_hour', 'Reg_ID'])
raw_file_ag = raw_file_ag.merge(raw_file_gr['total_amount'].median(), on=['tpep_pickup_hour', 'Reg_ID'])

# Агрегируем данные по часам и зонам поездок (по высадке пассажиров)
raw_file_add['dropoff'] = ''
dropoff_file_gr = raw_file_add.groupby(by = ['tpep_dropoff_hour', 'Reg_ID'], as_index=False)
dropoff_file_ag = dropoff_file_gr['dropoff'].count()
dropoff_file_ag.rename(columns = {'tpep_dropoff_hour': 'tpep_pickup_hour'}, inplace=True)
df = raw_file_ag.merge(dropoff_file_ag, on=['tpep_pickup_hour', 'Reg_ID'])
df.head()


,tpep_pickup_hour,Reg_ID,pickup,trip_distance,total_amount,dropoff
0,2016-05-01 00,1015,1,7.5,29.16,1
1,2016-05-01 00,1017,1,1.5,8.80,1
2,2016-05-01 00,1029,3,3.5,20.80,2
3,2016-05-01 00,1030,1,2.7,20.80,1
4,2016-05-01 00,1033,4,0.8,7.26,4


В настоящий момент в таблице только данные по тем зонам и часам, где и когда были поездки (сделано умышленно, чтобы не увеличивать объем таблицы раньше времени). Но также нам необходимо в таблицах иметь данные по зонам и часам, по которым не было поездок. Для этого сгенерируем все комбинации зон и часов и объединим с таблицей, содержащей информацию о кол-ве поездок. Пустые значения заменим на нули.

In [116]:
allcombinations = [[a, b] for a, b in itertools.product(np.arange(0, 2500, 1)+1, np.unique(df.tpep_pickup_hour))]
df_full = pd.DataFrame(allcombinations, columns=['Reg_ID', 'tpep_pickup_hour'])
df_ = df.merge(df_full, how='outer', on=['tpep_pickup_hour', 'Reg_ID'])
df_=df_.fillna(0)
df_['pickup'] = list(map(int, df_['pickup'])) # сконвертируем pickup в interger
df_['dropoff'] = list(map(int, df_['dropoff'])) # сконвертируем dropoff в interger
df_.sort_values(by='pickup', ascending=False, inplace=False).head() # отсортируем по убыванию кол-ва поездок

,tpep_pickup_hour,Reg_ID,pickup,trip_distance,total_amount,dropoff
11684,2016-05-03 20,1282,2206,1.80,11.800,2201
11512,2016-05-03 19,1282,2145,1.58,12.360,2132
15662,2016-05-04 20,1282,2078,1.80,12.300,2108
11858,2016-05-03 21,1282,2010,1.90,12.275,2099
101054,2016-05-24 19,1282,2009,1.66,12.950,2123


In [117]:
# запишем в отдельный файл результат агрегации
df_.to_csv('/Users/shrlq/Documents/python_yandex/course 6/week_6/agg_data_/2016-05_added.csv', ';')

### Построим график количества поездок жёлтого такси из района Empire State Building
А именно - из зоны, где находится Empire State Building, т.е. соответствующей координатам: 40.748603, -73.986482. Сначала узнаем идентификатор зоны, а потом отфильтруем по нему конечную таблицу с агрегированными данными и построим график по часам.

In [ ]:
long = -73.986482
latit = 40.748603

condition = sum([regions_data['west']<=long, regions_data['east']>=long, regions_data['south']<=latit, regions_data['north']>=latit]) == 4
regions_data.loc[condition, :]

In [ ]:
%matplotlib inline
intest_data = df.loc[df['Reg_ID'] == int(regions_data.loc[condition, 'region']), :] # в условии фильтра вводим идентификатор зоны
pl = intest_data.plot(x = 'tpep_pickup_hour', y='count', title='Поездки от Empire State Building в мае 2016',
                     label='Количество поездок', figsize=(12,5))
pl.set_xlabel('Час дня')
pl.set_ylabel('Количество поездок')
plt.show()

### Выясним, когда и где не было поездок
Посчитаем, сколько в мае 2016 было пар час-ячейка, для которых не было совершено ни одной поездки.

In [ ]:
df_empty = df.loc[df['count'] == 0, :]
df_empty.head()

In [ ]:
print ('Кол-во пар часа и зоны, когда не было поездок: ' + str(df_empty.shape[0]))